# Analysis Initialization and Functions

In [17]:
import plotly
from plotly import tools
import plotly.io as pio
import qcodes as qc
import numpy as np
from plotly import graph_objs as go
from qcodes.instrument_drivers.rohde_schwarz import SGS100A
from time import sleep
from qcodes.dataset.measurements import Measurement
import qcodes.dataset.experiment_container as exc
from scipy.optimize import curve_fit
import datetime
import scipy.fftpack
from scipy import signal
from scipy.signal import find_peaks

In [18]:
configuration = qc.config
print(f'Using config file from {configuration.current_config_path}')
configuration['core']['db_location'] = './2019-07-08-frays-final.db'
print(f'Database location: {configuration["core"]["db_location"]}')
qc.initialise_database()

Using config file from //anaconda3/lib/python3.7/site-packages/qcodes/config/qcodesrc.json
Database location: ./2019-07-08-frays-final.db


In [4]:
experimentsList = [exp.name for exp in exc.experiments()]
print(experimentsList)

['magFieldAlignment', 'ODMR_check', 'Alignement_Improvement', 'Calibration', 'overnight_Rabi', 'power_check_rabi', 'frequency_checks', 'overnight_peakonly_spinecho', 'overnight_peakonly_spinecho_for_real', 'pulsedOdmr_central_7', 'pulsedOdmr_rabi_central_7', 'pulsedOdmr_rabi_echo_weekend_7', 'pulsedOdmr_rabi_echo_postCooldown_70K', 'pulsedOdmr_rabi_echo_postCooldown_70K_forReal', 'pulsedOdmr_rabi_echo_postCooldown_70K_adjustedFreq', 'pulsedOdmr_rabi_echo_postCooldown_70K_Overnight', 'pulsedOdmr_rabi_echo_40K_Overnight', 'pulsedOdmr_rabi_echo_40K_check', 'rabi_echo_40K', 'cwODMR_15K', 'pulsedOdmr_rabi_echo_15K', 'pulsedOdmr_rabi_echo_15K_overnight', 'pulsedOdmr_rabi_echo_15K_final', 'pulsedOdmr_rabi_echo_2-5K', 'pulsedOdmr_rabi_echo_2K', 'pulsedOdmr_7NVs_2K', 'Odmr_7NVs_100K', 'pulsedOdmr_7NVs_100K', 'pulsedOdmr_rabi_spinecho_7NVs_100K', 'pulsedOdmr_rabi_spinecho_7NVs_100K_overnight', 'pulsedOdmr_rabi_spinecho_7NVs_40K', 'CWodmr_7NVs_40K', 'rabi_spinEcho_7NVs_40K', 'cwODMR_7NVs_55K', 'r

In [5]:
def normalizeCounts(countArray, num):
    reb_counts = np.squeeze(countArray)/np.mean(np.sort(np.squeeze(countArray))[-num:])
    return reb_counts

In [6]:
def lorentzian(x, x0, a0, g, amp):
    denom = (x - x0)**2 + (0.5*g)**2
    num = 0.5*g
    frac = a0 - (num/denom) * (amp)/np.pi
    return frac

In [7]:
def sinedamp(x,a,c,f,t):
    fun = a*np.cos(2*np.pi*f*x)*np.exp(-1*(x/t)) + c
    return fun

In [8]:
def stretchedexp(x,a,c,k,t):
    fun = a*np.exp(-1*(x/t)**k) + c
    return fun

### To fit the signal with a lorentzian function

In [9]:
def fitlorentzian(plotfun, expt='pulsedodmr', file=[], lb=None,ub=None):
    "Fitting parameters are ODMR frequency in GHz, Width in GHz, and Amplitude. \
    Attribute - expt - can either be 'odmr' or 'pulsedodmr'"
    if np.size(file) != 0:
        Data = exc.load_by_id(file[0])
    else:
        lastExperiment = exc.load_last_experiment()
        Data = lastExperiment.last_data_set()
    xaxis = 'Frequency'
    x1 = np.squeeze(Data.get_data(xaxis))
    
    if expt == 'odmr':
        yaxis = 'Counts'
        y1 = normalizeCounts(Data.get_data(yaxis),50)
    elif expt == 'pulsedodmr':
        y1 = np.squeeze(Data.get_data('Rebased_Counts'))  
        
    index = np.argmin(y1)
    freq = x1[index]/1e9
    
    if (lb and ub) == None:
        lb = [freq-0.01, 0.95, 0.007, 0.0005]
        ub = [freq+0.01, 1.1, 0.02, 0.005]
        
    popt, pcov = curve_fit(lorentzian, x1/1e9, y1, bounds=(lb, ub))
    fitname = 'Fit' + ', Resonance @ ' + str(round((popt[0]),3)) + ' GHz'
    plotfun.add_scatter(x = x1 , y = lorentzian(x1/1e9,*popt), name = fitname) 
    return popt

### To fit the signal with a damped sine function

In [10]:
def fitsinedamp(plotfun=None,file=[],expt='spinecho',lb=None,ub=None):   
    "Fitting parameters are Amplitude, Baseline Offset, Oscillation Frequency in GHz, and Decay Time in ns.\
    Attribute - expt - can either be 'rabi' or 'spinecho' or 'doubleecho'"
    if np.size(file) != 0:
        Data = exc.load_by_id(file[0])
    else:
        lastExperiment = exc.load_last_experiment()
        Data = lastExperiment.last_data_set()
    xaxis = 'Time'
    
    
    if expt == 'rabi':
        x1 = np.squeeze(Data.get_data(xaxis))
        y1 = np.squeeze(Data.get_data('Rebased_Counts'))
    elif expt == 'spinecho':
        x1 = 2*np.squeeze(Data.get_data(xaxis))
        y1 = np.squeeze(Data.get_data('Rebased_Counts'))
    elif expt == 'doubleecho':
        x1 = 2*np.squeeze(Data.get_data(xaxis))
        y1ms0 = np.squeeze(Data.get_data('Act_Counts'))
        y1ms1 = np.squeeze(Data.get_data('Ref_Counts'))
        y1 = y1ms0 - y1ms1
        
    ampMin = 0.8*(y1.max()-y1.min())/2
    ampMax = 1.2*(y1.max()-y1.min())/2
    boMin = 0.4*(y1.max()+y1.min())
    boMax = 0.6*(y1.max()+y1.min())
    freq = 1/(x1[y1.argmin()]*2)
    freqMin = freq*0.7
    freqMax = freq*1.3
    if (lb and ub) == None:
        lb = [ampMin, boMin, freqMin, 0]
        ub = [ampMax, boMax, freqMax, 1e5]
#     print(f'low bounds are: {lb}')
#     print(f'upper bounds are: {ub}')
    popt, pcov = curve_fit(sinedamp, x1, y1, bounds=(lb,ub))
    yval = sinedamp(x1,*popt)
    fitname = 'Fit' + ', \u0394 = ' + str(round((popt[0]*2*100),1)) + ' %' + ', \u03C0 = ' +  str(round((0.5/popt[2]),1)) + ' ns'
    if plotfun is not None:
        plotfun.add_scatter(x = x1 , y = yval, name = fitname,line=dict(shape='spline')) 
    return popt, pcov

### To fit the signal with a stretched exponential function

In [11]:
def find_T2(plotfun,file=[],expt='doubleecho',threshold = 0.45, lb=[0,0,0,0],ub=[2,3,3,5e6], revivals=0):   
    "Fitting parameters are Amplitude, Baseline Offset, Power of Stretched Exponential, and Decay Time in ns.\
    Attribute - expt - can either be 'spinecho' or 'doubleecho'.\
    threshold is required for peak detection"
    
    if np.size(file) != 0:
        Data = exc.load_by_id(file[0])
    else:
        lastExperiment = exc.load_last_experiment()
        Data = lastExperiment.last_data_set()
        
    x1 = 2*np.squeeze(Data.get_data('Time'))
    if expt == 'spinecho':
        y1 = np.squeeze(Data.get_data('Rebased_Counts'))  
    elif expt == 'doubleecho':
        y1ms0 = np.squeeze(Data.get_data('Act_Counts'))
        y1ms1 = np.squeeze(Data.get_data('Ref_Counts'))
        y1 = y1ms0 - y1ms1 
        y1 = (y1 + max(y1))/(2*max(y1))
    if revivals == 1:
        peaks, _= find_peaks(y1,height=threshold,distance=6, width=3)
        x0 = np.array([0])
        y0 = np.array([y1[0]])
        xpeaks = np.concatenate([x0, x1[peaks]], axis=0)
        print(xpeaks)
        ypeaks = np.concatenate([y0, y1[peaks]], axis=0)
        print(ypeaks)
    else:
        xpeaks = x1
        ypeaks = y1
    
    popt, pcov = curve_fit(stretchedexp, xpeaks, ypeaks, bounds=(lb,ub))
    yval = stretchedexp(xpeaks,*popt)
    
    fitname = 'Fit, T2 = ' + str(round((popt[3]/1e3),1)) + ' \u03BCs' 
    
    #plotfun.add_scatter(x = xpeaks, y = ypeaks, mode='markers', name = 'Detected Peaks',marker=dict(color='red', size=10, opacity=0.5)) 
    plotfun.add_scatter(x = xpeaks , y = yval, name = fitname, line=dict(shape='spline'), mode='lines')
    
    return popt, pcov

### To generate a  fourier transform

In [12]:
def fouriertransform(file=[],expt='spinecho'):
    "Attribute - expt - can either be 'rabi' or 'ramsey' or 'spinecho' or 'doubleecho'"
    layout = go.Layout(xaxis=dict(title='Frequency (GHz)'),yaxis=dict(title='Amplitude'),title = 'Fourier Transform of a ' + expt.capitalize() + ' signal')
    freqPlot = go.Figure(layout=layout)
    if np.size(file) != 0:
        Data = exc.load_by_id(file[0])
    else:
        lastExperiment = exc.load_last_experiment()
        Data = lastExperiment.last_data_set()
        
    xaxis = 'Time'
    if expt == 'rabi' or expt == 'ramsey':
        x1 = np.squeeze(Data.get_data(xaxis))
        y1 = np.squeeze(Data.get_data('Rebased_Counts'))
    elif expt == 'spinecho':
        x1 = 2*np.squeeze(Data.get_data(xaxis))
        y1 = np.squeeze(Data.get_data('Rebased_Counts'))
    elif expt == 'doubleecho':
        x1 = 2*np.squeeze(Data.get_data(xaxis))
        y1ms0 = np.squeeze(Data.get_data('Act_Counts'))
        y1ms1 = np.squeeze(Data.get_data('Ref_Counts'))
        y1 = y1ms0 - y1ms1
    elif expt == 'nmr':
        x1 = np.squeeze(Data.get_data(xaxis))
        y1ms0 = np.squeeze(Data.get_data('Act_Counts'))
        y1ms1 = np.squeeze(Data.get_data('Ref_Counts'))
        y1 = y1ms1 - y1ms0
        
    step = x1[1] - x1[0]
        
    fourTrans = np.fft.fft(y1)
    freqs = np.fft.fftfreq(y1.shape[-1], step)
    fourTransReal = fourTrans.real
    freqPlot.add_scatter(x = freqs, y = fourTransReal,line=dict(shape='spline'), mode='lines')
    return freqPlot

### To calculate and plot magnitude + direction of the applied field

In [13]:
def magfield(odmr1=[2.87],odmr2=[2.87],strain=5):
    "Input ODMR frequencies in GHz and strain splitting in MHz"
    
    g = 2.8024e6
    D = 2.877e9
    
    pts = np.size(odmr1)
    theta = np.zeros([pts])
    B0 = np.zeros([pts])
    
    for i in range (pts):
        n1 = odmr1[i]*1e9 + strain*1e6/2
        n2 = odmr2[i]*1e9 - strain*1e6/2
        P = n1**2 + n2**2 - n1*n2
        Q = (n1 + n2)*(2*n1**2 + 2*n2**2-5*n1*n2)
        B = np.sqrt((P-D**2)/3);
        c2t = (Q + 9*D*B**2+2*D**3)/(27*D*B**2);
        angle = np.arccos(np.sqrt(c2t))*180/np.pi;
        theta[i] = round(angle,2);
        B = B.real/g;
        B0[i] = round(B,2);
    
    fig = tools.make_subplots(rows=1, cols=2)
    plotfun = go.Figure(fig)
    plotfun.layout.yaxis.title = 'Magnetic Field Magnitude (gauss)'
    plotfun.layout.yaxis2.title = 'Angle to NV axis (degrees)'
    plotfun.add_bar(y=B0,row=1,col=1,name='Magnitude')
    plotfun.add_bar(y=theta,row=1,col=2,name='Angle')
    plotfun.layout.title = 'Checking magnetic field alignment'
    return plotfun

### To plot recent and saved data

In [24]:
def plotdata(expt='odmr', plotcurrent=0, normalize= 1, files=[1], nPi = []): 
    "Attribute - expt - can either be 'counting' or 'odmr' or 'pulsedodmr' or 'rabi' or 'ramsey' or 'spinecho' or 'doubleecho' or 'nmr'.\
    If value of plotcurrent is 1 then it will plot the current data. If value of normalize is 1 then it will normalize 'odmr' and 'doubleecho' signal."
    if plotcurrent == 1:        
        filesize = np.size(files) + 1
    else:
        filesize = np.size(files)  
        
    plotfun = go.Figure()    
    for i in range(filesize):
        if plotcurrent == 1 and i == filesize-1:
            Data2 = exc.load_last_experiment()
            Data2 = Data2.last_data_set()
        else:    
            Data2 = exc.load_by_id(files[i])
        if expt == 'spinecho':
            x2 = 2*np.squeeze(Data2.get_data('Time'))
            y2 = np.squeeze(Data2.get_data('Rebased_Counts'))
            plotfun.layout = go.Layout(xaxis=dict(title='Time (ns)'),yaxis=dict(title='Counts'),title=expt.capitalize())                  
        elif expt == 'doubleecho':
            if nPi == [] or nPi[i] == 1:
                x2 = 2*np.squeeze(Data2.get_data('Time'))
            else:
                x2 = nPi[i]*np.squeeze(Data2.get_data('Time'))
            y2ms0 = np.squeeze(Data2.get_data('Act_Counts'))
            y2ms1 = np.squeeze(Data2.get_data('Ref_Counts'))
            y2 = y2ms0 - y2ms1   
            plotfun.layout = go.Layout(xaxis=dict(title='Time (ns)'),yaxis=dict(title='Counts'),title='Spinecho Double Measure')
            if normalize == 1:
                y2 = (y2 + max(y2))/(2*max(y2))
        elif expt == 'nmr':
            x2 = np.squeeze(Data2.get_data('Time'))
            y2ms0 = np.squeeze(Data2.get_data('Act_Counts'))
            y2ms1 = np.squeeze(Data2.get_data('Ref_Counts'))
            y2 = y2ms0 - y2ms1   
            plotfun.layout = go.Layout(xaxis=dict(title='Time (ns)'),yaxis=dict(title='Counts'),title='NMR')
            if normalize == 1:
                y2 = (y2 + max(y2))/(2*max(y2))
        elif expt == 'odmr':
            x2 = np.squeeze(Data2.get_data('Frequency'))
            y2 = np.squeeze(Data2.get_data('Counts'))
            plotfun.layout = go.Layout(xaxis=dict(title='Frequency (Hz)'),yaxis=dict(title='Counts'),title=expt.upper())
            if normalize == 1:
                y2 = normalizeCounts(Data2.get_data('Counts'),50)
                layout = go.Layout(xaxis=dict(title='Frequency (Hz)'),yaxis=dict(title='Normalized Counts'),title=expt.upper())
        elif expt == 'pulsedodmr':
            x2 = np.squeeze(Data2.get_data('Frequency'))
            y2 = np.squeeze(Data2.get_data('Rebased_Counts'))   
            plotfun.layout = go.Layout(xaxis=dict(title='Frequency (Hz)'),yaxis=dict(title='Counts'),title='Pulsed ODMR')
        elif expt == 'g2':
            x2 = np.squeeze(Data2.get_data('Time'))
            y2 = np.squeeze(Data2.get_data('Norm_Counts'))
            plotfun.layout = go.Layout(xaxis=dict(title='Time dif'), yaxis=dict(title='Normalised Counts'), title='g2 Dip')
        else:
            x2 = np.squeeze(Data2.get_data('Time'))
            y2 = np.squeeze(Data2.get_data('Rebased_Counts'))     
            plotfun.layout = go.Layout(xaxis=dict(title='Time (ns)'),yaxis=dict(title='Counts'),title=expt.capitalize())
        if plotcurrent == 1 and i == filesize-1:
            plotfun.add_scatter(x = x2, y = y2, name = 'Recent Data', mode='lines+markers') 
        else:       
            plotfun.add_scatter(x = x2, y = y2, name = files[i], mode='lines+markers') 
    return plotfun

### To update recent data

In [15]:
def updatedata(plotfun, expt='rabi', sleeptime=2, normalize=1, animation=1):
    "Attribute - expt - can either be 'counting' or 'odmr' or 'pulsedodmr' or rabi' or 'ramsey' or 'spinecho' or 'doubleecho'.\
    If value of plotcurrent is 1 then it will plot the current data. If value of normalize is 1 then it will normalize 'odmr' and 'doubleecho' signal."
    for i in range(10000):
        sleep(sleeptime)
        if animation == 1:
            with plotfun.batch_animate(easing = 'quad'):   
                Data2 = exc.load_last_experiment()
                Data2 = Data2.last_data_set()
                if expt == 'doubleecho':
                    y2ms0 = np.squeeze(Data2.get_data('Act_Counts'))
                    y2ms1 = np.squeeze(Data2.get_data('Ref_Counts'))
                    y2 = y2ms0 - y2ms1  
                    if normalize == 1:
                        y2 = (y2 + max(y2))/(2*max(y2))
                elif expt == 'nmr':
                    y2ms0 = np.squeeze(Data2.get_data('Act_Counts'))
                    y2ms1 = np.squeeze(Data2.get_data('Ref_Counts'))
                    y2 = y2ms0 - y2ms1  
                    if normalize == 1:
                        y2 = (y2 + max(y2))/(2*max(y2))
                elif expt == 'odmr':
                    y2 = np.squeeze(Data2.get_data('Counts'))
                    if normalize == 1:
                        y2 = normalizeCounts(Data2.get_data('Counts'),50)
                elif expt == 'g2':
                    y2 = np.squeeze(Data2.get_data('Norm_Counts'))
                else:
                    y2 = np.squeeze(Data2.get_data('Rebased_Counts'))     
                plotfun.data[-1].y = y2
        else:
            Data2 = exc.load_last_experiment()
            Data2 = Data2.last_data_set()
            if expt == 'doubleecho':
                y2ms0 = np.squeeze(Data2.get_data('Act_Counts'))
                y2ms1 = np.squeeze(Data2.get_data('Ref_Counts'))
                y2 = y2ms0 - y2ms1  
                if normalize == 1:
                    y2 = (y2 + max(y2))/(2*max(y2))
            elif expt == 'nmr':
                y2ms0 = np.squeeze(Data2.get_data('Act_Counts'))
                y2ms1 = np.squeeze(Data2.get_data('Ref_Counts'))
                y2 = y2ms0 - y2ms1  
                if normalize == 1:
                    y2 = (y2 + max(y2))/(2*max(y2))
            elif expt == 'odmr':
                y2 = np.squeeze(Data2.get_data('Counts'))
                if normalize == 1:
                    y2 = normalizeCounts(Data2.get_data('Counts'),50)
            elif expt == 'g2':
                y2 = np.squeeze(Data2.get_data('Counts'))
            else:
                y2 = np.squeeze(Data2.get_data('Rebased_Counts'))     
            plotfun.data[-1].y = y2

In [25]:
plotdata()